# Analysis of gyms in Nuremberg, Germany
This notebook is part of the Capstone Project of the Coursera Course <b>Applied Data Science Capstone</b>.

In [1]:
import numpy as np
import pandas as pd
import requests

!conda install -c conda-forge geopy --yes # uncomment this line when not already installed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line when not already installed
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.1
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.11.8  |       ha878542_0         145 KB  conda-forge
    certifi-2020.11.8          |   py36h5fab9bb_0         150 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         392 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch

# Get center coordinates for Nuremberg

In [2]:
address = 'Nürnberg, DE'

geolocator = Nominatim(user_agent="de_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))

lat_nbg = location.latitude
lng_nbg = location.longitude

The geograpical coordinates of Nürnberg, DE are 49.453872, 11.077298.


# Configure foursquare

In [3]:
CLIENT_ID = '1ASA1R5SJNI4KKZDE0VT02FRVCZPTLGRT35LWIICH2VGSSYW' # your Foursquare ID
CLIENT_SECRET = 'AQEUCQQX3WF40RDYNTMNFCVRNNV5OY1CC3MCVOSD212UVEKL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1ASA1R5SJNI4KKZDE0VT02FRVCZPTLGRT35LWIICH2VGSSYW
CLIENT_SECRET:AQEUCQQX3WF40RDYNTMNFCVRNNV5OY1CC3MCVOSD212UVEKL


# Get the gyms in Nuremberg - via direct search from center coordinates
Approach: search for all "Gym/Fitness Center" within a radius from city center 

In [4]:
def getNearbyCategory(name, latitude, longitude, categorylist, radius):

# using the foursquare API...
# function to get the venues from a categorylist (can also contain only one value) that are within
# a certain radius of a point definded by name, latitude and longitude
# name is only used for having it in the returned dataframe. it is not needed for the foursquare quiery

    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, categorylist, radius, LIMIT)

    results = requests.get(url).json()["response"]['venues']

    venues_list = []
    venues_list.append([(
            name, 
            latitude, 
            longitude,
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name'],
            v['id']
            ) for v in results])
    venues_list= venues_list[0]
    nearby_venues = pd.DataFrame(data=venues_list)

    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue Id']
    return(nearby_venues)

In [5]:
# instead of using "query" which matches venue name, use "CategoryId" which matches for a (list of) categories!

name = "Nürnberg"
lat = lat_nbg # 49.453872
lng = lng_nbg # 11.077298

search_category = '4bf58dd8d48988d175941735' # category ID for "Gym/Fitness Center"
radius = 5000  

nbg_gyms = getNearbyCategory(name, lat, lng, search_category, radius)   
nbg_gyms

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Id
0,Nürnberg,49.453872,11.077298,FIT ONE,49.449467,11.074092,Gym / Fitness Center,586ca64e18dc536fecf25667
1,Nürnberg,49.453872,11.077298,FIT STAR Fitnessstudio Nürnberg-Zentrum,49.448212,11.083224,Gym / Fitness Center,577d6a44498eadda3146ab2d
2,Nürnberg,49.453872,11.077298,Schwimmbad Ebensee,49.467580,11.139481,Gym Pool,52dd6b8c498e35e77df9f275
3,Nürnberg,49.453872,11.077298,FIT STAR Fitnessstudio Fürth-Südstadt,49.459940,11.006402,Gym / Fitness Center,577d68b0498eea8e9db20e71
4,Nürnberg,49.453872,11.077298,Kieser Training,49.444779,11.109701,Gym / Fitness Center,5310437c11d2a01b7c45b5b6
5,Nürnberg,49.453872,11.077298,Post SV Fitlounge,49.453875,11.089603,Gym / Fitness Center,4b7d849af964a520e1c32fe3
6,Nürnberg,49.453872,11.077298,BodytecPoint,49.471080,11.022999,Gym / Fitness Center,52652161498efba58d4440e0
7,Nürnberg,49.453872,11.077298,clever fit Fitnessstudio Nürnberg,49.423984,11.068681,Gym,50a2da52f2e7ce3e7a58855e
8,Nürnberg,49.453872,11.077298,Fitness First Lifestyle Club,49.450252,11.070769,Gym / Fitness Center,4bd9a7ece914a593034557fa
9,Nürnberg,49.453872,11.077298,Powerbox Bei Eugen,49.455033,11.037993,Gym / Fitness Center,5a7593a4e179101a278e3371


In [27]:
# visualize the gyms on the map
# create map
map_nbg = folium.Map(location=[lat_nbg, lng_nbg], zoom_start=12)

# add the gyms as blue circle markers
for lat, lng, label in zip(nbg_gyms["Venue Latitude"], nbg_gyms["Venue Longitude"], nbg_gyms["Venue"]):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(map_nbg)

#display map    
map_nbg

# Get the ratings of the gyms

In [57]:
# cell for quickly checking the forsquare response manually... if nothing is returned check result variable for 'quota exceeded'
venue_id = nbg_gyms.loc[8,'Venue Id']
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
#result['response']['venue']
try: 
            data_available = result['response']['venue']
except:
            print(result['meta']['errorDetail'])
#print(result['meta']['errorDetail'])

Quota exceeded


In [7]:
def getRatingData(venue_ids):
# for a given venue_id list, this fuction extracts, for each venue in the list, the following data from Foursquare:
# Rating, Number of Tips, Number of Likes, Number of Dislikes
# if no rating data is available Nan is written
# a Dataframe with the rating info is returned
# if no data is returned from Foursquare, this function returns the error message 
    
    rating_info=[]
    for v_id in venue_ids:
        print(v_id)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
            v_id, 
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
            
        # make the GET request
        result = requests.get(url).json()
       
        # check if data is there, if not return with error message
        try: 
            data_available = result['response']['venue']
        
        except:
            error_msg = result['meta']['errorDetail']
            return(error_msg)
        
        # extract needed info from data
        else:
            try:
                rating = result['response']['venue']['rating']
            except:
                rating = np.nan
            
            # get number of tips
            nr_tips = result['response']['venue']['tips']['count']
       
            try:
                nr_likes = result['response']['venue']['likes']['count']
            except:
                nr_likes = np.nan
            
            # get number of dislikes
            try:
                nr_dislikes = result['response']['venue']['dislike']['count']
            except:
                nr_dislikes = np.nan
    
            rating_info.append([
                v_id,
                rating, 
                nr_tips, 
                nr_likes,
                nr_dislikes]) 

            rating_data = pd.DataFrame(data=rating_info)
            rating_data.columns = ['Venue Id', 'Rating', "No of Tips", "No of Likes", "No of Dislikes"] 
              
    return(rating_data)
           

In [8]:
gym_ratings = getRatingData(nbg_gyms["Venue Id"])
#gym_ratings

586ca64e18dc536fecf25667
577d6a44498eadda3146ab2d
52dd6b8c498e35e77df9f275
577d68b0498eea8e9db20e71
5310437c11d2a01b7c45b5b6
4b7d849af964a520e1c32fe3
52652161498efba58d4440e0
50a2da52f2e7ce3e7a58855e
4bd9a7ece914a593034557fa
5a7593a4e179101a278e3371
4ea3ca007ee52a35dd77700a
4e81eb795503ca7789259b2e
51670c17e4b02703e1e4a095
4cb5e85b64998cfa706c11a2
4b99429df964a520676e35e3
542e69d4498efe79608418b0
523348a611d2d7333803e886
50705bf6e4b0d7b67eed8a0a
4e823e8d4901000af3a71620
5837521eba883710b2c85ecc
50d36db8e4b0abb8c47f790f
580cb4ed38fa79522b4ce4e2
51d5973b498e8230f9a7f24b
524d0e53498e841847909a16
58c6ff90d25ded210443be22
4fa10232e4b07aaa44dee796
4f02d449c2ee2cc871333ab5
4b7980cef964a52036fd2ee3
4cc74269be87468842747daf
4b71aa3af964a5209d542de3
4eba6334dab495868c4d79ab
4e336d6d52b11c7462230567
4e560919c65be3cea46095cd
50b21e79e4b0ed0fc18a17f6
4e335b03d22d3229f59f2ccf
509946b2498e20f69f8a1b88
55537586498e0e84c620535f
50925fe4e4b0b512e250feb6
509d120be4b0787347cd89b4
4eafbb0c9a521bacdb65c7ba


In [9]:
gym_ratings

,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes
0,586ca64e18dc536fecf25667,7.8,1,8,NaN
1,577d6a44498eadda3146ab2d,NaN,0,1,NaN
2,52dd6b8c498e35e77df9f275,NaN,0,1,NaN
3,577d68b0498eea8e9db20e71,NaN,0,1,NaN
4,5310437c11d2a01b7c45b5b6,NaN,0,1,NaN
5,4b7d849af964a520e1c32fe3,NaN,2,5,NaN
6,52652161498efba58d4440e0,NaN,0,1,NaN
7,50a2da52f2e7ce3e7a58855e,7.3,2,6,NaN
8,4bd9a7ece914a593034557fa,8.6,3,34,NaN
9,5a7593a4e179101a278e3371,NaN,0,0,NaN


In [12]:
print("Minimum Number of Likes: ", gym_ratings["No of Likes"].min())
print("Maximum Number of Likes: ", gym_ratings["No of Likes"].max())
gym_ratings[gym_ratings["No of Likes"]>0]

Minimum Number of Likes:  0
Maximum Number of Likes:  51


,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes
0,586ca64e18dc536fecf25667,7.8,1,8,NaN
1,577d6a44498eadda3146ab2d,NaN,0,1,NaN
2,52dd6b8c498e35e77df9f275,NaN,0,1,NaN
3,577d68b0498eea8e9db20e71,NaN,0,1,NaN
4,5310437c11d2a01b7c45b5b6,NaN,0,1,NaN
5,4b7d849af964a520e1c32fe3,NaN,2,5,NaN
6,52652161498efba58d4440e0,NaN,0,1,NaN
7,50a2da52f2e7ce3e7a58855e,7.3,2,6,NaN
8,4bd9a7ece914a593034557fa,8.6,3,34,NaN
11,4e81eb795503ca7789259b2e,NaN,0,3,NaN


In [35]:
print("Minimum Rating: ", gym_ratings.Rating.min())
print("Maximum Rating: ", gym_ratings.Rating.max())
print(gym_ratings[gym_ratings["Rating"]>0].shape)
gym_ratings[gym_ratings["Rating"]>0]

Minimum Rating:  -1.0
Maximum Rating:  8.9
(11, 6)
(32, 6)


,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes,Cluster
0,586ca64e18dc536fecf25667,7.8,1,8,-1.0,3
7,50a2da52f2e7ce3e7a58855e,7.3,2,6,-1.0,3
8,4bd9a7ece914a593034557fa,8.6,3,34,-1.0,1
12,51670c17e4b02703e1e4a095,7.9,6,15,-1.0,2
14,4b99429df964a520676e35e3,6.7,1,8,-1.0,3
25,4fa10232e4b07aaa44dee796,8.7,3,20,-1.0,2
26,4f02d449c2ee2cc871333ab5,8.4,8,51,-1.0,1
27,4b7980cef964a52036fd2ee3,7.6,3,20,-1.0,2
29,4b71aa3af964a5209d542de3,8.2,0,18,-1.0,2
30,4eba6334dab495868c4d79ab,7.6,2,13,-1.0,2


In [41]:
print(gym_ratings[gym_ratings["Rating"]<0].shape)
gym_ratings[(gym_ratings["Rating"] < 0) & (gym_ratings["No of Likes"] > 0)]

(32, 6)


,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes,Cluster
1,577d6a44498eadda3146ab2d,-1.0,0,1,-1.0,0
2,52dd6b8c498e35e77df9f275,-1.0,0,1,-1.0,0
3,577d68b0498eea8e9db20e71,-1.0,0,1,-1.0,0
4,5310437c11d2a01b7c45b5b6,-1.0,0,1,-1.0,0
5,4b7d849af964a520e1c32fe3,-1.0,2,5,-1.0,0
6,52652161498efba58d4440e0,-1.0,0,1,-1.0,0
11,4e81eb795503ca7789259b2e,-1.0,0,3,-1.0,0
13,4cb5e85b64998cfa706c11a2,-1.0,0,4,-1.0,0
16,523348a611d2d7333803e886,-1.0,0,3,-1.0,0
18,4e823e8d4901000af3a71620,-1.0,0,1,-1.0,0


In [13]:
print("Minimum Number of Tips: ", gym_ratings["No of Tips"].min())
print("Maximum Number of Tips: ", gym_ratings["No of Tips"].max())
gym_ratings[gym_ratings["No of Tips"]>0]

Minimum Number of Tips:  0
Maximum Number of Tips:  8


,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes
0,586ca64e18dc536fecf25667,7.8,1,8,NaN
5,4b7d849af964a520e1c32fe3,NaN,2,5,NaN
7,50a2da52f2e7ce3e7a58855e,7.3,2,6,NaN
8,4bd9a7ece914a593034557fa,8.6,3,34,NaN
12,51670c17e4b02703e1e4a095,7.9,6,15,NaN
14,4b99429df964a520676e35e3,6.7,1,8,NaN
20,50d36db8e4b0abb8c47f790f,NaN,3,5,NaN
25,4fa10232e4b07aaa44dee796,8.7,3,20,NaN
26,4f02d449c2ee2cc871333ab5,8.4,8,51,NaN
27,4b7980cef964a52036fd2ee3,7.6,3,20,NaN


# Cluster gyms according to their rating data

In [14]:
# replace Nan with  -1 as Kmeans can't handle Nans
gym_ratings = gym_ratings.fillna(-1)

In [15]:
# set number of clusters
kclusters = 4

gym_ratings_clustering = gym_ratings.drop(['Venue Id','No of Dislikes'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0)
kmeans.fit(gym_ratings_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([3, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 1, 2, 0, 2], dtype=int32)

In [16]:
gym_ratings["Cluster"] = kmeans.labels_

Cluster 0: "No rating available"

In [17]:
print((gym_ratings[gym_ratings["Cluster"] == 0]).shape)
gym_ratings[gym_ratings["Cluster"] == 0]

(32, 6)


,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes,Cluster
1,577d6a44498eadda3146ab2d,-1.0,0,1,-1.0,0
2,52dd6b8c498e35e77df9f275,-1.0,0,1,-1.0,0
3,577d68b0498eea8e9db20e71,-1.0,0,1,-1.0,0
4,5310437c11d2a01b7c45b5b6,-1.0,0,1,-1.0,0
5,4b7d849af964a520e1c32fe3,-1.0,2,5,-1.0,0
6,52652161498efba58d4440e0,-1.0,0,1,-1.0,0
9,5a7593a4e179101a278e3371,-1.0,0,0,-1.0,0
10,4ea3ca007ee52a35dd77700a,-1.0,0,0,-1.0,0
11,4e81eb795503ca7789259b2e,-1.0,0,3,-1.0,0
13,4cb5e85b64998cfa706c11a2,-1.0,0,4,-1.0,0


Cluster 1: "high rating and high number of likes"

In [18]:
gym_ratings[gym_ratings["Cluster"] == 1]

,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes,Cluster
8,4bd9a7ece914a593034557fa,8.6,3,34,-1.0,1
26,4f02d449c2ee2cc871333ab5,8.4,8,51,-1.0,1
42,4e5f9e30ae600b89d7c62f65,8.9,7,42,-1.0,1


Cluster 2: "medium number of likes"

In [19]:
gym_ratings[gym_ratings["Cluster"] == 2]

,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes,Cluster
12,51670c17e4b02703e1e4a095,7.9,6,15,-1.0,2
25,4fa10232e4b07aaa44dee796,8.7,3,20,-1.0,2
27,4b7980cef964a52036fd2ee3,7.6,3,20,-1.0,2
29,4b71aa3af964a5209d542de3,8.2,0,18,-1.0,2
30,4eba6334dab495868c4d79ab,7.6,2,13,-1.0,2


Cluster 3: "medium rating, low number of likes"

In [20]:
gym_ratings[gym_ratings["Cluster"] == 3]

,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes,Cluster
0,586ca64e18dc536fecf25667,7.8,1,8,-1.0,3
7,50a2da52f2e7ce3e7a58855e,7.3,2,6,-1.0,3
14,4b99429df964a520676e35e3,6.7,1,8,-1.0,3


In [21]:
# merge rating and cluster info back in nbg_gyms dataframe
nbg_gyms = pd.merge(nbg_gyms, gym_ratings, how='left', on='Venue Id')

In [42]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow = ['#8000ff', '#2adddd', '#006400', '#ff0000'] # manually overwritten to make cluster 3 visible on the map, olive green was hard to see

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nbg_gyms['Venue Latitude'], nbg_gyms['Venue Longitude'], nbg_gyms['Venue'], nbg_gyms['Cluster']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
#nbg_gyms["No of Likes"] == nbg_gyms["No of Likes"].max()
nbg_gyms[nbg_gyms["No of Likes"] == nbg_gyms["No of Likes"].max()]

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes,Cluster
26,Nürnberg,49.453872,11.077298,McFIT,49.459672,11.022928,Gym / Fitness Center,4f02d449c2ee2cc871333ab5,8.4,8,51,-1.0,1


# Investigation about different results with different query radius

In [43]:
# using different radius seems to lead to not only more/less but DIFFERENT results. Check this

name = "Nürnberg"
lat = lat_nbg # 49.453872
lng = lng_nbg # 11.077298

search_category = '4bf58dd8d48988d175941735' # category ID for "Gym/Fitness Center"
radius = [500, 1000, 2000, 3000, 4000, 5000] 

df_ids = pd.DataFrame()
for rad in radius: 
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_category, rad, LIMIT)
    
    results = requests.get(url).json()["response"]['venues']

    venues_list = []
    venues_list.append([(
            v['name']#, v['id']
            ) for v in results])
    venues_list= venues_list[0]
    #print(len(venues_list))
    padding = (50-len(venues_list)) * [0]  # attention: 50 hardcoded means the maximum amount of venues returned for a radius must not be bigger than 50 
    venues_list = venues_list + padding
    column = "radius"+str(rad)
    df_ids[column] = venues_list

df_ids 

,radius500,radius1000,radius2000,radius3000,radius4000,radius5000
0,FIT ONE,FIT ONE,FIT ONE,FIT ONE,FIT ONE,FIT ONE
1,Peter Althof Boxgym,FIT STAR Fitnessstudio Nürnberg-Zentrum,FIT STAR Fitnessstudio Nürnberg-Zentrum,FIT STAR Fitnessstudio Nürnberg-Zentrum,FIT STAR Fitnessstudio Nürnberg-Zentrum,FIT STAR Fitnessstudio Nürnberg-Zentrum
2,Zumbafit,Post SV Fitlounge,Jumpers Fitness,Jumpers Fitness,Jumpers Fitness,Schwimmbad Ebensee
3,Bodyvision,Fitness First Lifestyle Club,Kieser Training,Kieser Training,BodytecPoint,FIT STAR Fitnessstudio Fürth-Südstadt
4,"Zumba-Studio ""Move It""",DAS STUDIO – Aerobic in der City,Fitness First Lifestyle Club,Post SV Fitlounge,Kieser Training,Fitness First Lifestyle Club
5,0,Novotel Gym & Spa,Post SV Fitlounge,Kieser Training,Post SV Fitlounge,clever fit Fitnessstudio Nürnberg
6,0,Figura,DAS STUDIO – Aerobic in der City,Fitness First Lifestyle Club,Fitness First Lifestyle Club,BodytecPoint
7,0,Fitnessstudio,Lady Style,Powerbox Bei Eugen,Kieser Training,Kieser Training
8,0,RUN Nürnberg | Stadtlauf,Sibeliusbad,Lady Style,clever fit Fitnessstudio Nürnberg,Post SV Fitlounge
9,0,Fitness & Wellness NH Hotel,Space Yoga,DAS STUDIO – Aerobic in der City,Powerbox Bei Eugen,Powerbox Bei Eugen


In [44]:
name = "Nürnberg"
lat = lat_nbg # 49.453872
lng = lng_nbg # 11.077298

search_category = '4bf58dd8d48988d175941735' # category ID for "Gym/Fitness Center" 

df500 = getNearbyCategory(name, lat, lng, search_category, 500).sort_values(by="Venue Id")
df1000 = getNearbyCategory(name, lat, lng, search_category, 1000).sort_values(by="Venue Id")
df2000 = getNearbyCategory(name, lat, lng, search_category, 2000).sort_values(by="Venue Id")
df3000 = getNearbyCategory(name, lat, lng, search_category, 3000).sort_values(by="Venue Id")
df4000 = getNearbyCategory(name, lat, lng, search_category, 4000).sort_values(by="Venue Id")
df5000 = getNearbyCategory(name, lat, lng, search_category, 5000).sort_values(by="Venue Id")

In [45]:
df500venues = df500["Venue Id"].to_list() 
df1000venues = df1000["Venue Id"].to_list() 
df2000venues = df2000["Venue Id"].to_list() 
df3000venues = df3000["Venue Id"].to_list() 
df4000venues = df4000["Venue Id"].to_list() 
df5000venues = df5000["Venue Id"].to_list() 

In [46]:
# I assume that every venue that is within a radius of 500 is also within a radius of 1000. Let's check if this is true
print("Are all venues from radius=500 also in radius=1000? ", set(df500venues).intersection(df1000venues) == set(df500venues))
# I assume that every venue that is within a radius of 500 is also within a radius of 2000. Let's check if this is true
print("Are all venues from radius=500 also in radius=2000? ",set(df500venues).intersection(df2000venues) == set(df500venues))
print("Number of venues from radius=500 that are also in radius=2000: ", len(set(df500venues).intersection(df2000venues)))
# only 2 venues from the 500 radius are also in the 2000 radius, isnt this strange? let's check for 1000 vs 2000
print("Are all venues from radius=1000 also in radius=2000? ",set(df1000venues).intersection(df2000venues) == set(df1000venues))
print("Number of venues from radius=1000 that are also in radius=2000: ", len(set(df1000venues).intersection(df2000venues)))

Are all venues from radius=500 also in radius=1000?  True
Are all venues from radius=500 also in radius=2000?  False
Number of venues from radius=500 that are also in radius=2000:  2
Are all venues from radius=1000 also in radius=2000?  False
Number of venues from radius=1000 that are also in radius=2000:  14


So I can conclude that depending on the radius given in the search query Foursquare returns different venues. I assumed that it would just return more venues with an increasing radius. But this is not the case. 